## BIOS 470/570 Lecture 17

## Last time we covered:
* ### Introduction to biological imaging
* ### FIJI/ImageJ
* ### Reading and displaying images in python

## Today we will cover:
* ### Image arithmetic
* ### Morphological operations

In [ ]:
import numpy as np
from skimage import io, exposure, util, morphology
import matplotlib.pyplot as plt
from matplotlib import animation, rc

# default colormap, figure size, and font size
plt.rcParams['image.cmap']='gray'
plt.rcParams['figure.figsize']=(8,8)
plt.rcParams['font.size'] = 24

### We will work with an example file which has 3 channels and 3 timepoints:

In [ ]:
img = io.imread('data/Example.tif',plugin='tifffile')
img.shape

### Using the first (timepoint) index, gives us the whole stack at that timepoint:

In [ ]:
img1 = img[0]
img2 = img[1]
img1.shape

### Let's define some functions to save time later:

In [ ]:
def showNoAxis(img):
    """display an image and turn off the axis"""
    plt.imshow(img)
    plt.axis('off')

In [ ]:
def make8bit(img):
    """Convert an image to 8-bit by scaling its values in the 1st-99th percentile to the range 0,255"""
    img_8bit = img
    for cc in range(img.shape[2]):
        Imin, Imax = np.percentile(img[:,:,cc],(1, 99))
        img_8bit[:,:,cc] = exposure.rescale_intensity(img[:,:,cc],in_range=(Imin, Imax), out_range=(0,255))
    return img_8bit

In [ ]:
def show8bit(img):
    """Convert an image to 8-bit and display it with no axis"""
    img_8bit = make8bit(img.astype(int))
    showNoAxis(img_8bit)

### We can display the RGB version of the image if we convert to 8-bit

In [ ]:
show8bit(img1)

### Let's see how we can break this image into individual channels and then put them back together. 

In [ ]:
img1_chan1 = img1[:,:,0]
img1_chan2 = img1[:,:,1]

img1_merged = io.concatenate_images((img1_chan1,img1_chan2))
img1_merged.shape

### This has merged them along the first axis, but for channels to display correctly as RGB, they need to be the third axis (first one is good for time or z dimension). We can fix this with the move axis function:

In [ ]:
img1_merged = np.moveaxis(img1_merged,[0,1,2],[2,0,1]) # make axis 0 into 2, 1 into 0, and 2 into 1
img1_merged.shape

### This is the right axes but for RGB we need 3. We can add an array of zeros to create an empty channel and allow this to be displayed:

In [ ]:
zz = np.zeros(img1_chan1.shape)
zz.shape

### np.dstack works along the third dimension:

In [ ]:
img1_merged = np.dstack((img1_merged,zz))

In [ ]:
show8bit(img1_merged)

### Some other possible color choices we could make just by shifting around the channels:

In [ ]:
show8bit(np.dstack((zz,img1_chan1,img1_chan2)))

In [ ]:
show8bit(np.dstack((img1_chan1,zz,img1_chan2)))

In [ ]:
show8bit(np.dstack((img1_chan1,img1_chan1,img1_chan2)))

In [ ]:
show8bit(np.dstack((img1_chan2,img1_chan1,img1_chan2)))

### These images are 16 bit integer arrays and this affects what happens when you do arithmetic with them:

In [ ]:
showNoAxis(img1_chan1);

In [ ]:
showNoAxis(img1_chan1*10)

In [ ]:
showNoAxis(img1_chan1*1000)

### What happened? We reached the limit on the 16bit integers. Even the background reaches the max now, and no image is visible:

In [ ]:
img1_chan1.max()

In [ ]:
(img1_chan1*1000).max()

### Some operations will result in conversions to float and the order in which you do them will matter:

In [ ]:
img_scale = (2**16-1)*img1_chan1/img1_chan1.max();
showNoAxis(img_scale)

In [ ]:
img_scale.max()

In [ ]:
img_scale = (img1_chan1/img1_chan1.max())*(2**16-1);
showNoAxis(img_scale);


In [ ]:
img_scale.max()

### We see that this is no longer an integer array. We can turn it back into one with astype. Specifically, this is for 16 bit integers:

In [ ]:
img_scale = img_scale.astype('uint16')

In [ ]:
img_scale.max()

### We can avoid the automatic conversions and confusing code that comes with them by doing the conversion to float ourselves before doing the arithmetic. skimage has a set of tools for this in the util module:

In [ ]:
img_float = util.img_as_float(img1_chan1)

### This scales the image range so that the total possible range (0 to 2^16-1 for 16 bit) scales onto the interval from 0 to 1:

In [ ]:
img_float.max()

In [ ]:
img_float = img_float/img_float.max()*(2**16-1)

In [ ]:
showNoAxis(img_float)

### Binary masks can be used to identify features in images. Here we look at a simple thresholding:

In [ ]:
showNoAxis(img1_chan2)

In [ ]:
img_binary = img1_chan2 > 800

In [ ]:
showNoAxis(img_binary)

### Basic morphological operations: erosion and dilation. 

In [ ]:
img_dilate = morphology.dilation(img_binary,morphology.disk(4))
showNoAxis(img_dilate)

In [ ]:
img_erode = morphology.erosion(img_binary,morphology.disk(4))
showNoAxis(img_erode)

### skimage has various functions to define the neighborhood for the morphological operation:

In [ ]:
morphology.disk(4)

In [ ]:
morphology.square(4)

In [ ]:
img_dilate = morphology.dilation(img_binary,morphology.square(10))
showNoAxis(img_dilate)

### Let's see how these work on an image with a single bright pixel:

In [ ]:
img1pix = np.zeros((256,256))
img1pix[128,128] = 1
showNoAxis(img1pix)

In [ ]:
showNoAxis(morphology.dilation(img1pix,morphology.disk(10)))

In [ ]:
showNoAxis(morphology.dilation(img1pix,morphology.square(10)))

In [ ]:
showNoAxis(morphology.dilation(img1pix,morphology.diamond(10)))

### Compound operations. Opening and closing:

#### Opening: erosion followed by dilation. 

In [ ]:
img_open = morphology.opening(img_binary,morphology.disk(8))
fig = plt.figure(figsize=(16,8))
fig.add_subplot(1,2,1)
showNoAxis(img_binary)
fig.add_subplot(1,2,2)
showNoAxis(img_open)

#### Closing: dilation followed by erosion:

In [ ]:
img_close = morphology.closing(img_binary,morphology.disk(8))
fig = plt.figure(figsize=(16,8))
fig.add_subplot(1,2,1)
showNoAxis(img_binary)
fig.add_subplot(1,2,2)
showNoAxis(img_close)

### You can use overlays into RGB images to see the effects of these operations

In [ ]:
fig = plt.figure(figsize=(16,8))
fig.add_subplot(1,2,1)
showNoAxis(np.dstack((img_binary,img_open,zz)))
plt.title('Opening')
fig.add_subplot(1,2,2)
showNoAxis(np.dstack((img_binary,img_close,zz)))
plt.title('Closing');


### Morphological operations can also be performed on intensity images:

In [ ]:
img_dilate = morphology.opening(img1_chan2,morphology.disk(8))
fig = plt.figure(figsize=(16,8))
fig.add_subplot(1,2,1)
showNoAxis(img1_chan2)
fig.add_subplot(1,2,2)
showNoAxis(img_dilate)